## Display Features

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

from xml.etree.ElementTree import fromstring, ElementTree
import xml.etree.ElementTree as ET

In [2]:
%matplotlib inline
pd.options.display.max_rows = 300000
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 500

# Load Dataset

In [3]:
football = sqlite3.connect(
    "C:\\Users\\Space\\Documents\\py\\Projects\\TuringCollege\\Football\\DataSet\\database.sqlite"
)

# List of Tables

In [4]:
football_db = {}

result = football.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in result:
    query = "SELECT * from " + str(table[0]) + ";"
    football_db[table[0]] = pd.read_sql_query(query, football)

football_db.keys()

dict_keys(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match', 'League', 'Country', 'Team', 'Team_Attributes'])

# Country

In [5]:
country = football_db["Country"].copy()

In [6]:
country.rename(columns={"id": "CountryID", "name": "CountryName"}, inplace=True)

In [7]:
country

,CountryID,CountryName
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [8]:
country.shape

(11, 2)

# League

In [9]:
league = football_db["League"].copy()

In [10]:
del league["id"]

In [11]:
league.rename(columns={"country_id": "CountryID", "name": "LeagueName"}, inplace=True)

In [12]:
league

,CountryID,LeagueName
0,1,Belgium Jupiler League
1,1729,England Premier League
2,4769,France Ligue 1
3,7809,Germany 1. Bundesliga
4,10257,Italy Serie A
5,13274,Netherlands Eredivisie
6,15722,Poland Ekstraklasa
7,17642,Portugal Liga ZON Sagres
8,19694,Scotland Premier League
9,21518,Spain LIGA BBVA


In [13]:
league.shape

(11, 2)

# Player

In [263]:
player = football_db["Player"].copy()

In [264]:
player["birthday"] = pd.DatetimeIndex(player["birthday"]).year

In [265]:
player.drop(columns=["id", "player_fifa_api_id"], inplace=True)

In [266]:
player.rename(
    columns={
        "player_api_id": "PlayerID",
        "player_name": "PlayerName",
        "birthday": "Birthday",
        "height": "Height",
        "weight": "Weight",
    },
    inplace=True,
)

In [269]:
player["BMI"] = (player["Weight"] / 2.205) / ((player["Height"] / 100.0) ** 2)

In [270]:
player.head()

,PlayerID,PlayerName,Birthday,Height,Weight,BMI
0,505942,Aaron Appindangoye,1992,182.88,187,25.357159
1,155782,Aaron Cresswell,1989,170.18,146,22.862685
2,162549,Aaron Doran,1991,170.18,163,25.524778
3,30572,Aaron Galindo,1982,182.88,198,26.848756
4,23780,Aaron Hughes,1979,182.88,154,20.882366


In [15]:
player.shape

(11060, 7)

# Team

In [20]:
team = football_db["Team"].copy()

In [21]:
team.drop(
    columns=[
        "id",
        "team_fifa_api_id",
    ],
    inplace=True,
)

In [22]:
team.rename(
    columns={
        "team_api_id": "TeamID",
        "team_long_name": "TeamLongName",
        "team_short_name": "TeamShortName",
    },
    inplace=True,
)

In [23]:
team.head()

,TeamID,TeamLongName,TeamShortName
0,9987,KRC Genk,GEN
1,9993,Beerschot AC,BAC
2,10000,SV Zulte-Waregem,ZUL
3,9994,Sporting Lokeren,LOK
4,9984,KSV Cercle Brugge,CEB


In [24]:
team.shape

(299, 3)

# Player Attributes

In [14]:
player_attributes = football_db["Player_Attributes"].copy()

In [15]:
player_attributes.drop(
    columns=[
        "id",
        "player_fifa_api_id",
        "gk_diving",
        "gk_handling",
        "gk_kicking",
        "gk_positioning",
        "gk_reflexes",
    ],
    inplace=True,
)

In [16]:
def break_ties(col: str):
    most_common = pd.Series.mode(col)

    if len(most_common) > 1:
        return most_common[0]
    else:
        return most_common

In [ ]:
player_attributes_cat = (
    player_attributes.groupby(["player_api_id", "date"])[
        ["preferred_foot", "attacking_work_rate", "defensive_work_rate"]
    ]
    .agg(lambda x: break_ties(x))
    .reset_index()
)

In [7]:
player_attributes.rename(
    columns={
        "player_api_id": "PlayerID",
        "date": "Date",
        "overall_rating": "OverallRating",
        "potential": "Potential",
        "preferred_foot": "PreferredFoot",
        "attacking_work_rate": "AttachWorkRate",
        "defensive_work_rate": "DefenceWorkRate",
        "crossing": "Crossing",
        "finishing": "Finishing",
        "heading_accuracy": "HeadingAccuracy",
        "short_passing": "ShortPassing",
        "volleys": "Volleys",
        "dribbling": "Dribbling",
        "curve": "Curve",
        "free_kick_accuracy": "FreeKickAccuracy",
        "long_passing": "LongPassing",
        "ball_control": "BallControl",
        "acceleration": "Acceleration",
        "sprint_speed": "SprintSpeed",
        "agility": "Agility",
        "reactions": "Reactions",
        "balance": "Balance",
        "shot_power": "ShotPower",
        "jumping": "Jumping",
        "stamina": "Stamina",
        "strength": "Strength",
        "long_shots": "LongShots",
        "aggression": "Aggression",
        "interceptions": "Interceptions",
        "positioning": "Positioning",
        "vision": "Vision",
        "penalties": "Penalties",
        "marking": "Marking",
        "standing_tackle": "StandingTackle",
        "sliding_tackle": "SlidingTackle",
    },
    inplace=True,
)

In [8]:
player_attributes["Date"] = pd.DatetimeIndex(player_attributes["Date"]).year

In [9]:
null_instances = player_attributes[player_attributes["OverallRating"].isnull()].index

In [10]:
player_attributes.drop(null_instances, inplace=True)

In [11]:
player_attributes.shape

(183142, 35)

In [12]:
player_attributes.isnull().sum()

PlayerID               0
Date                   0
OverallRating          0
Potential              0
PreferredFoot          0
AttachWorkRate      2394
DefenceWorkRate        0
Crossing               0
Finishing              0
HeadingAccuracy        0
ShortPassing           0
Volleys             1877
Dribbling              0
Curve               1877
FreeKickAccuracy       0
LongPassing            0
BallControl            0
Acceleration           0
SprintSpeed            0
Agility             1877
Reactions              0
Balance             1877
ShotPower              0
Jumping             1877
Stamina                0
Strength               0
LongShots              0
Aggression             0
Interceptions          0
Positioning            0
Vision              1877
Penalties              0
Marking                0
StandingTackle         0
SlidingTackle       1877
dtype: int64

In [243]:
df_inner = pd.merge(player, player_attributes, on="PlayerID", how="inner")

In [244]:
df_inner.head()

,PlayerID,PlayerName,Birthday,Height,Weight,Date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
0,505942,Aaron Appindangoye,1992,182.88,187,2016,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
1,505942,Aaron Appindangoye,1992,182.88,187,2015,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
2,505942,Aaron Appindangoye,1992,182.88,187,2015,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0
3,505942,Aaron Appindangoye,1992,182.88,187,2015,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0
4,505942,Aaron Appindangoye,1992,182.88,187,2007,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0


# Team Attributes

In [213]:
team_attributes = football_db["Team_Attributes"].copy()

In [214]:
team_attributes.shape

(1458, 25)

In [215]:
team_attributes.drop(
    columns=[
        "id",
        "team_fifa_api_id",
        "buildUpPlayDribbling",
        "buildUpPlaySpeedClass",
        "buildUpPlayDribblingClass",
        "buildUpPlayPassingClass",
        "buildUpPlayPositioningClass",
        "chanceCreationPassingClass",
        "chanceCreationCrossingClass",
        "chanceCreationShootingClass",
        "chanceCreationPositioningClass",
        "defencePressureClass",
        "defenceAggressionClass",
        "defenceTeamWidthClass",
        "defenceDefenderLineClass",
    ],
    inplace=True,
)

In [217]:
team_attributes.rename(
    columns={
        "team_api_id": "TeamID",
        "date": "Date",
        "buildUpPlaySpeed": "BuildUpPlaySpeed",
        "buildUpPlayPassing": "BuildUpPlayPassing",
        "chanceCreationPassing": "ChanceCreationPassing",
        "chanceCreationCrossing": "ChanceCreationCrossing",
        "chanceCreationShooting": "ChanceCreationShooting",
        "defencePressure": "DefencePressure",
        "defenceAggression": "DefenceAggression",
        "defenceTeamWidth": "DefenceTeamWidth",
    },
    inplace=True,
)

In [218]:
team_attributes["Date"] = pd.DatetimeIndex(team_attributes["Date"]).year

In [219]:
team_attributes.head()

,TeamID,Date,BuildUpPlaySpeed,BuildUpPlayPassing,ChanceCreationPassing,ChanceCreationCrossing,ChanceCreationShooting,DefencePressure,DefenceAggression,DefenceTeamWidth
0,9930,2010,60,50,60,65,55,50,55,45
1,9930,2014,52,56,54,63,64,47,44,54
2,9930,2015,47,54,54,63,64,47,44,54
3,8485,2010,70,70,70,70,70,60,70,70
4,8485,2011,47,52,53,48,52,47,47,52


# Match

In [177]:
def extract_xml(row, col_name, xml_key, away_home):
    count = 0

    element = row[col_name]
    team_id = row[away_home + "_team_api_id"]

    if type(element) == int:
        return element

    elif element != None:
        # print(row,element)
        tree = ElementTree(fromstring(element))
        root = tree.getroot()

        for child in root.iter(xml_key):
            if str(team_id) == child.text:
                count += 1
        return count
    else:
        return np.nan

In [190]:
def extract_possession_xml(row, col_name, xml_key):
    count = 0
    sum_pos = 0

    element = row[col_name]

    if type(element) == int:
        return element

    elif element != None:
        # print(row,element)
        tree = ElementTree(fromstring(element))
        root = tree.getroot()
        for child in root.iter(xml_key):
            count += 1
            sum_pos += int(child.text)

        if count == 0:
            return np.nan
        else:
            return sum_pos / count
    else:
        return np.nan

In [174]:
match = football_db["Match"].copy()

In [175]:
match["TotalGoals"] = np.nan

In [176]:
match["TotalGoals"] = match["home_team_goal"] + match["away_team_goal"]

In [178]:
match["ShotonHome"] = np.nan
match["ShotonAway"] = np.nan

In [179]:
match["ShotonHome"] = match.apply(
    lambda x: extract_xml(x, "shoton", "team", "home"), axis=1
)
match["ShotonAway"] = match.apply(
    lambda x: extract_xml(x, "shoton", "team", "away"), axis=1
)

In [180]:
match["ShotoffHome"] = np.nan
match["ShotoffAway"] = np.nan

In [181]:
match["ShotoffHome"] = match.apply(
    lambda x: extract_xml(x, "shotoff", "team", "home"), axis=1
)
match["ShotoffAway"] = match.apply(
    lambda x: extract_xml(x, "shotoff", "team", "away"), axis=1
)

In [182]:
match["FoulCommitHome"] = np.nan
match["FoulCommitAway"] = np.nan

In [183]:
match["FoulCommitHome"] = match.apply(
    lambda x: extract_xml(x, "foulcommit", "team", "home"), axis=1
)
match["FoulCommitAway"] = match.apply(
    lambda x: extract_xml(x, "foulcommit", "team", "away"), axis=1
)

In [184]:
match["CardHome"] = np.nan
match["CardAway"] = np.nan

In [185]:
match["CardHome"] = match.apply(
    lambda x: extract_xml(x, "card", "team", "home"), axis=1
)
match["CardAway"] = match.apply(
    lambda x: extract_xml(x, "card", "team", "away"), axis=1
)

In [186]:
match["CrossHome"] = np.nan
match["CrossAway"] = np.nan

In [187]:
match["CrossHome"] = match.apply(
    lambda x: extract_xml(x, "cross", "team", "home"), axis=1
)
match["CrossAway"] = match.apply(
    lambda x: extract_xml(x, "cross", "team", "away"), axis=1
)

In [188]:
match["CornerHome"] = np.nan
match["CornerAway"] = np.nan

In [189]:
match["CornerHome"] = match.apply(
    lambda x: extract_xml(x, "corner", "team", "home"), axis=1
)
match["CornerAway"] = match.apply(
    lambda x: extract_xml(x, "corner", "team", "away"), axis=1
)

In [191]:
match["PossessionHome"] = np.nan
match["PossessionAway"] = np.nan

In [192]:
match["PossessionHome"] = match.apply(
    lambda x: extract_possession_xml(x, "possession", "homepos"), axis=1
)
match["PossessionAway"] = match.apply(
    lambda x: extract_possession_xml(x, "possession", "awaypos"), axis=1
)

In [193]:
match.drop(
    columns=["id", "stage", "country_id"],
    inplace=True,
)

In [194]:
match.rename(
    columns={
        "league_id": "LeagueID",
        "match_api_id": "MatchID",
        "home_team_api_id": "HomeTeamID",
        "away_team_api_id": "AwayTeamID",
        "date": "Date",
        "season": "Season",
        "home_team_goal": "HomeTeamGoal",
        "away_team_goal": "AwayTeamGoal",
    },
    inplace=True,
)

In [ ]:
match.head()

In [196]:
# match.drop(match.iloc[:, 8:112], inplace = True, axis = 1)

In [197]:
match["Date"] = pd.DatetimeIndex(match["Date"]).year

In [198]:
match.head()

,LeagueID,Season,Date,MatchID,HomeTeamID,AwayTeamID,HomeTeamGoal,AwayTeamGoal,TotalGoals,ShotonHome,ShotonAway,ShotoffHome,ShotoffAway,FoulCommitHome,FoulCommitAway,CardHome,CardAway,CrossHome,CrossAway,CornerHome,CornerAway,PossessionHome,PossessionAway
0,1,2008/2009,2008,492473,9987,9993,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2008/2009,2008,492474,10000,9994,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2008/2009,2008,492475,9984,8635,0,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2008/2009,2008,492476,9991,9998,5,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2008/2009,2008,492477,7947,9985,1,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
